In [ ]:
import requests
import pandas as pd
import json

url = "https://jiji.ng/api_web/v1/listing?slug=houses-apartments-for-rent&init_page=true&region_slug=lagos&page=200&webp=true&po=13.2.2.3.i&lsmid=1723725178722"

headers = {
    'Cookie': 'app=009fb9ad8ac543209f8b921333396efe; first_visit=1473229581; rid=direct; uid=66969f5b158f3aa1ca2dcc8fee12e953d6b3430d'
}

# Here i sent the GET request
response = requests.get(url, headers=headers)

# Checked if the request was successful
if response.status_code == 200:
    # Parse the response to JSON
    data = response.json()

    # you can option this out
    #  #Pretty-print the JSON data
    print(json.dumps(data, indent=4))

else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")



In [ ]:
# here i defined the Function to fetch data from a specific page
def fetch_page_data(page_number):
    url = f"https://jiji.ng/api_web/v1/listing?slug=houses-apartments-for-rent&init_page=true&region_slug=lagos&page={page_number}&webp=true&po=13.2.2.3.i&lsmid=1723725178722"
    headers = {
        'Cookie': 'app=009fb9ad8ac543209f8b921333396efe; first_visit=1473229581; rid=direct; uid=66969f5b158f3aa1ca2dcc8fee12e953d6b3430d'
    }
    response = requests.get(url, headers=headers)
    return response.json()

# Empty list to store all listings
all_listings = []

# I Looped through multiple pages to collect listings and print my process
for page in range(1, 301):  # Fetches the total pages available to me, you can get this from postman
    print(f"Fetching page {page}...")
    data = fetch_page_data(page)
    
    # Here i Checked if there are adverts in the response
    adverts = data.get("adverts_list", {}).get("adverts", [])
    if not adverts:  # Break the loop if no adverts are found
        break
    
    # Extract and store the listing data
    for advert in adverts:
        advert_info = {
            "Title": advert.get("title"),
            "Price": advert.get("price_title"),
            "Region": advert.get("region"),
            "Short Description": advert.get("short_description"),
            "Type of Apartment": advert.get("title").split()[1]  # Extracting apartment type from title
        }

        # Extract additional attributes (like property size, bedrooms, bathrooms, etc.)
        for attr in advert.get("attrs", []):
            advert_info[attr["name"]] = f'{attr["value"]} {attr.get("unit", "")}'.strip()

        # Append the advert info to the listings list
        all_listings.append(advert_info)

# Convert the aggregated data into a DataFrame
df = pd.DataFrame(all_listings)

# Display the DataFrame
print(df.head())

# Optional: Save the DataFrame to a CSV file
df.to_csv("jiji_all_listings.csv", index=False)

#Note: you can actually use this same code for multiple cities accross Nigeria.


In [ ]:
#This is an extra cleaning step to get the house type from the Title column of our dataframe

# Sample list of house types i got from the website
house_name_list= [
"Apartment", 
"Block of Flats",
"Duplex",
"Mini Flat",
"Townhouse",
"Terrace",
"Bedsitter",
"Bungalow",
"Chalet",
"Condo",
"Farm House",
"House",
"Maisonette",
"Mansion",
"Penthouse",
"Room & Parlour",
"Shared Apartment",
"Studio Apartment",
"Villa"]

# Load the CSV file into a DataFrame
df = pd.read_csv(r'jiji_all_listings.csv', encoding='ISO-8859-1')

# Function to find the house type in the description
def find_house_type(Title):
    for house_type in house_name_list:
        if house_type.lower() in Title.lower():
            return house_type
    return None

# Apply the function to the 'description' column and create a new 'house_type' column
df['house_type'] = df['Title'].apply(find_house_type)

# Save the updated DataFrame back to a CSV
df.to_csv('Lagos_file.csv', index=False)

